# RAG 통합 최종 버전
## 다현님 + 나현님 + 사용자 작업 통합

이 노트북은 세 분의 작업을 통합한 최종 버전입니다:
- **다현님 작업**: 본문/부칙 분리, 학위/정원 문장 생성, 본문 청킹
- **나현님 작업**: UpstageDocumentParseLoader 활용 (참고)
- **사용자 작업**: 표 데이터 처리 및 벡터화

### 통합 전략
1. PDF 본문과 부칙을 분리하여 청킹
2. CSV 표 데이터를 문장으로 변환
3. 모든 문서를 FAISS 벡터 스토어에 통합
4. RAG 파이프라인 구축 및 평가


In [1]:
# 환경 설정
import os
import sys
from pathlib import Path
from typing import List, Optional

import pandas as pd
import re

# Upstage API Key 설정
os.environ.setdefault("UPSTAGE_API_KEY", "up_g01lLro3nefoqy9IZGw3NK4ExjfIQ")
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY", "")

# 작업 디렉토리 설정
CURRENT_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
if Path.cwd() != CURRENT_DIR:
    os.chdir(CURRENT_DIR)


print(f"✅ 작업 디렉토리: {CURRENT_DIR}")
print(f"✅ Upstage API Key: {UPSTAGE_API_KEY[:20]}...")


✅ 작업 디렉토리: c:\Users\user\-NLP-RAG-Project
✅ Upstage API Key: up_g01lLro3nefoqy9IZ...


In [2]:
# 필요한 라이브러리 설치
%pip install langchain_upstage langchain_community langchain-openai openai pdfplumber pandas faiss-cpu wikipedia-api


Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

PDF_PATH = CURRENT_DIR / "ewha.pdf"

if not PDF_PATH.exists():
    raise FileNotFoundError(f"❌ PDF 파일을 찾을 수 없습니다: {PDF_PATH}")

print("📄 PDF 파일 로딩 중...")
loader = PyPDFLoader(str(PDF_PATH))
docs = loader.load()

# 부칙 시작 전까지만 선택 (38page 기준)
docs = [d for d in docs if int(d.metadata["page"]) < 38]

print(f"✅ 본문 페이지 로드 완료: {len(docs)} 페이지")

full_text = "\n".join(d.page_content for d in docs)


# -------------------------------------------------------------
# 부칙("부칙") 등장 인덱스 찾기
# -------------------------------------------------------------
split_index = full_text.find("부칙")

if split_index == -1:
    print("⚠️ '부칙' 문자열을 찾을 수 없습니다. 전체 본문만 처리합니다.")
    main_text = full_text
    appendix_text = ""
else:
    main_text = full_text[:split_index]
    appendix_text = full_text[split_index:]

print(f"📄 본문 길이: {len(main_text)}")
print(f"📄 부칙 길이: {len(appendix_text)}")


# =============================================================
# 2. 본문 1차 구조 기반 분리
# =============================================================
def split_by_structure(text: str):
    """
    이화 학칙 문서를 조항/항목 기반으로 1차 분리
    """
    pattern = r"""
        (?=제\d+조)               # 제1조
        |(?=\n\d+\.)              # 1.
        |(?=\n\d+\))              # 1)
        |(?=\n\d+\s*-\s*\d+\s*-\s*\d+)  # 2 - 2 - 39
    """
    parts = re.split(pattern, text, flags=re.VERBOSE)
    return [p.strip() for p in parts if p.strip()]


structured_sections = split_by_structure(main_text)
print("📌 본문 1차 구조 분리 수:", len(structured_sections))


# =============================================================
# 3. 본문 RecursiveCharacterTextSplitter 청킹
# =============================================================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=200,
    separators=["별표", "\n\n제", "\n제", "\n\n", "\n", " "]  # 우선순위 높은 순
)

main_chunks = []
for sec in structured_sections:
    split_docs = text_splitter.create_documents([sec])
    main_chunks.extend(split_docs)

print("📌 최종 본문 청크 수:", len(main_chunks))


# =============================================================
# 4. 부칙 청킹
# =============================================================

# --- 4-1) 부칙을 '부칙(YYYY.MM.DD 개정)' 단위로 분리
def split_appendix(appendix: str):
    pattern = r"(?=부칙\(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\s*개정\))"
    parts = re.split(pattern, appendix)
    return [p.strip() for p in parts if p.strip()]

appendix_sections = split_appendix(appendix_text)
print("📌 1차 부칙 개정 단위 수:", len(appendix_sections))


# --- 4-2) 각 부칙 내 조항 기반 추가 분리
def split_by_articles(text: str):
    pattern = r"(?=제\d+조)|(?=\n?\d+\s*\()"
    parts = re.split(pattern, text)
    return [p.strip() for p in parts if p.strip()]

fine_grained_sections = []
for sec in appendix_sections:
    fine_grained_sections.extend(split_by_articles(sec))

print("📌 2차 부칙 조항 분리 수:", len(fine_grained_sections))


# --- 4-3) 최종 부칙 청킹
appendix_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " "]
)

appendix_chunks = appendix_splitter.create_documents(fine_grained_sections)

print("📌 최종 부칙 청크 수:", len(appendix_chunks))


# =============================================================
# 5. 결과 요약
# =============================================================
print("\n================ RESULT ================")
print(f"본문 청크 수: {len(main_chunks)}")
print(f"부칙 청크 수: {len(appendix_chunks)}")
print(f"총 청크 수: {len(main_chunks) + len(appendix_chunks)}")
print("========================================")

📄 PDF 파일 로딩 중...
✅ 본문 페이지 로드 완료: 38 페이지
📄 본문 길이: 20919
📄 부칙 길이: 20222
📌 본문 1차 구조 분리 수: 223
📌 최종 본문 청크 수: 223
📌 1차 부칙 개정 단위 수: 80
📌 2차 부칙 조항 분리 수: 250
📌 최종 부칙 청크 수: 251

================ RESULT ================
본문 청크 수: 223
부칙 청크 수: 251
총 청크 수: 474


In [4]:
# ==================================================================
# 📦 CSV 처리 전체 파이프라인 (VS Code 상대경로 기준)
# ==================================================================

import pandas as pd
import re
from pathlib import Path
from langchain.docstore.document import Document

CURRENT_DIR = Path.cwd()

# ==================================================================
# 1) degrees 폴더 → 학위 문장 생성
# ==================================================================

DEG_DIR = CURRENT_DIR / "degrees"

degree_files = [
    DEG_DIR / "degree_2006.csv의 사본",
    DEG_DIR / "degree_2009.csv의 사본",
    DEG_DIR / "degree_2017.csv의 사본",
    DEG_DIR / "degree_latest.csv의 사본",
]

degree_sentences = []

for path in degree_files:
    match = re.search(r"(20\d{2})", str(path))
    year = f"{match.group(1)} 개정" if match else "최신 개정"

    df = pd.read_csv(path, encoding="utf-8-sig")

    for _, row in df.iterrows():
        college = row.get("대학")
        degree = row.get("학사")
        major = row.get("전공")

        if pd.isna(college) or pd.isna(degree) or pd.isna(major):
            continue

        s = f"{str(college).strip()}의 {str(major).strip()} 전공은 {str(degree).strip()} 학위를 수여한다. ({year})"
        degree_sentences.append(s)

# 저장
out_degree = DEG_DIR / "degree_sentences_all.csv"
pd.DataFrame({"sentence": degree_sentences}).to_csv(out_degree, index=False, encoding="utf-8-sig")

# 1) degree_sentences → Document로 변환
degree_documents = [
    Document(
        page_content=s,
        metadata={"source": "[별표 2] 학사학위", "type": "degree"}
    )
    for s in degree_sentences
]
print(f"🎓 degree 문서 생성: {len(degree_documents)}개")

# ==================================================================
# 2) capacity.csv → 입학정원 문서 생성
# ==================================================================

csv_documents = []

cap_path = CURRENT_DIR / "capacity.csv"
df_cap = pd.read_csv(cap_path, encoding="utf-8-sig")

years = sorted(df_cap["year"].unique())

def build_quota_text(row):
    return f"""
    {int(row['year'])}학년도 입학정원 정보:
    - 대학: {row['college']}
    - 학부/학과: {row['department']}
    - 전공: {row['major']}
    - 정원: {row['quota']}명
    """.strip()

for year in years:
    df_year = df_cap[df_cap["year"] == year]
    for _, row in df_year.iterrows():
        text = build_quota_text(row)
        csv_documents.append(Document(
            page_content=text,
            metadata={
                "source": f"[별표 1] {year}학년도 입학정원",
                "type": "quota"
            }
        ))

print(f"🎉 capacity 문서 수: {len(csv_documents)}개")


# ==================================================================
# 3) contract_dept.csv → 계약학과 문서
# ==================================================================

def build_contract_text(row):
    return f"""
    계약학과 설치·운영 정보:
    - 설치대학: {row.get('설치대학','')}
    - 설치형태: {row.get('설치형태','')}
    - 학과/전공: {row.get('학과_전공','')}
    - 수여 학위: {row.get('학위_종류','')}
    - 입학 정원: {row.get('입학정원_명','')}명
    - 설치·운영 기간: {row.get('설치_운영기간','')}
    """.strip()

contract_path = CURRENT_DIR / "contract_dept.csv"
if contract_path.exists():
    df_ct = pd.read_csv(contract_path, encoding="utf-8-sig")
    for _, row in df_ct.iterrows():
        csv_documents.append(Document(
            page_content=build_contract_text(row),
            metadata={"source": "[별표 3] 계약학과", "type": "contract"}
        ))
    print(f"🎉 계약학과 문서: {len(df_ct)}개")
else:
    print("⚠ contract_dept.csv 없음")


# ==================================================================
# 4) grade.csv → GPA 변환표 문서
# ==================================================================

grade_documents = []

grade_path = CURRENT_DIR / "grade.csv"
df_grade = pd.read_csv(grade_path, encoding="utf-8-sig")

def build_gpa_text(row):
    return f"""
    GPA 변환 기준:
    - 기준 연도: {row['year']}
    - 등급: {row['grade']}
    - 환산 점수: {row['gpa']}
    """.strip()

for _, row in df_grade.iterrows():
    grade_documents.append(Document(
        page_content=build_gpa_text(row),
        metadata={"source": "GPA 변환표", "type": "grade"}
    ))

print(f"🎉 GPA 문서 생성: {len(grade_documents)}개")


# ==================================================================
# 최종 출력
# ==================================================================

print("\n📦 CSV 기반 총 Document 수:",
      len(degree_sentences) + len(csv_documents) + len(grade_documents))


🎓 degree 문서 생성: 160개
🎉 capacity 문서 수: 626개
🎉 계약학과 문서: 1개
🎉 GPA 문서 생성: 26개

📦 CSV 기반 총 Document 수: 813


In [5]:
# ==================================================================
# 3. 모든 문서 통합 및 FAISS 벡터 스토어 생성 (리팩토링 버전)
# ==================================================================

from pathlib import Path
import platform, tempfile
from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS

# -------------------------------------------------------
# 1) Upstage 임베딩 로드
# -------------------------------------------------------
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="solar-embedding-1-large"
)
print("✅ Upstage 임베딩 모델 로드 완료")


# -------------------------------------------------------
# 2) 문서 통합 (PDF 본문 + 부칙 + CSV + GPA)
# -------------------------------------------------------
all_documents = main_chunks + appendix_chunks  + degree_documents + csv_documents + grade_documents
print(f"📦 총 문서(청크) 수: {len(all_documents)}개")


# -------------------------------------------------------
# 3) 본문/부칙 metadata 정리
# -------------------------------------------------------
for doc in main_chunks:
    doc.metadata = doc.metadata or {}
    doc.metadata.update({"source": "ewha.pdf 본문", "type": "main_text"})

for doc in appendix_chunks:
    doc.metadata = doc.metadata or {}
    doc.metadata.update({"source": "ewha.pdf 부칙", "type": "appendix_text"})


# -------------------------------------------------------
# 4) 빈 문서 제거
# -------------------------------------------------------
valid_documents = [d for d in all_documents if d.page_content and d.page_content.strip()]
print(f"🧹 유효 문서 수: {len(valid_documents)}개")


# -------------------------------------------------------
# 5) 문서 타입별 통계 출력
# -------------------------------------------------------
type_stats = {}
for d in valid_documents:
    t = d.metadata.get("type", "unknown")
    type_stats[t] = type_stats.get(t, 0) + 1

print("\n📊 문서 타입 통계:")
for k, v in type_stats.items():
    print(f"  - {k}: {v}개")


# -------------------------------------------------------
# 6) 벡터 DB 저장 경로 설정
#     → Windows에서 한글 경로 문제 해결을 위해 TEMP 사용
# -------------------------------------------------------
if platform.system() == "Windows":
    base_dir = Path(tempfile.gettempdir()) / "rag_vectorstore"
    base_dir.mkdir(exist_ok=True)
    print(f"📂 Windows TEMP 디렉토리 사용: {base_dir}")
else:
    base_dir = CURRENT_DIR / "vectorstore"
    base_dir.mkdir(exist_ok=True)

vector_db_path = str(base_dir.resolve())


# -------------------------------------------------------
# 7) 벡터 스토어 생성/로드
# -------------------------------------------------------
FORCE_REBUILD = True

if not FORCE_REBUILD:
    try:
        vector_store = FAISS.load_local(
            folder_path=vector_db_path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True,
        )
        print("📂 기존 벡터스토어 로드 완료")
    except:
        print("⚠ 기존 벡터스토어 로드 실패 → 새로 생성")
        FORCE_REBUILD = True

if FORCE_REBUILD:
    print(f"\n🔄 벡터 스토어 새로 생성 중... (총 {len(valid_documents)}개 문서)")
    vector_store = FAISS.from_documents(valid_documents, embeddings)
    vector_store.save_local(vector_db_path)
    print("✅ 벡터스토어 저장 완료")

print(f"\n🎯 최종 문서 수: {len(valid_documents)}개")

✅ Upstage 임베딩 모델 로드 완료
📦 총 문서(청크) 수: 1287개
🧹 유효 문서 수: 1287개

📊 문서 타입 통계:
  - main_text: 223개
  - appendix_text: 251개
  - degree: 160개
  - quota: 626개
  - contract: 1개
  - grade: 26개
📂 Windows TEMP 디렉토리 사용: C:\Users\user\AppData\Local\Temp\rag_vectorstore

🔄 벡터 스토어 새로 생성 중... (총 1287개 문서)
✅ 벡터스토어 저장 완료

🎯 최종 문서 수: 1287개


In [6]:
import wikipediaapi
from langchain_text_splitters import RecursiveCharacterTextSplitter

def retrieve_wikipedia_docs(query: str, lang: str = 'en', chunk_size: int = 1000, chunk_overlap: int = 100):
    docs = []
    try:
        wiki_wiki = wikipediaapi.Wikipedia(
            language=lang,
            extract_format=wikipediaapi.ExtractFormat.WIKI,
            user_agent="NLP_Project"
        )
        
        page_py = wiki_wiki.page(query)
        
        if page_py.exists() and page_py.text:
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap
            )
            chunks = text_splitter.split_text(page_py.text)
            
            for i, chunk in enumerate(chunks):
                if chunk and chunk.strip():
                    docs.append(Document(
                        page_content=chunk,
                        metadata={
                            "source": "wikipedia",
                            "title": page_py.title,
                            "chunk_index": i,
                            "type": "wikipedia"
                        }
                    ))
    except:
        pass
    
    return docs
def extract_wikipedia_keywords(question: str) -> List[str]:
    """
    통합 업그레이드 버전:
    - 고유명사 추출
    - 영어 기반 MMLU 지식 키워드 추출
    - 한국어 분야 매핑
    - Law / Psych / Philosophy / History 분야 자동 감지
    - 대표 Wikipedia 문서 자동 매핑
    """

    keywords = []
    q = question.lower()

    # ----------------------------------------------------
    # 1) 고유명사(인물/사건/개념) 추출 (가장 우선순위 높음)
    # ----------------------------------------------------
    proper_nouns = re.findall(r"\b[A-Z][a-zA-Z]+\b", question)
    keywords.extend(proper_nouns)

    # ----------------------------------------------------
    # 2) 기존 academic_keywords (영어 기반 매핑)
    # ----------------------------------------------------
    academic_keywords = {
        'homo': 'Human evolution',
        'sapiens': 'Human evolution',
        'australopithecus': 'Human evolution',
        'erectus': 'Human evolution',
        'habilis': 'Human evolution',
        'bipolar': 'Psychology',
        'disorder': 'Psychology',
        'manic': 'Psychology',
        'kohlberg': 'Moral development',
        'kant': 'Philosophy',
        'crime': 'Criminology',
        'aristotle': 'Philosophy',
        'jurisprudence': 'Law',
        'ethics': 'Ethics',
        'larceny': 'Law',
        'criminal': 'Law',
        'clemenceau': 'History',
        'world war': 'History',
        'du fu': 'Literature',
        'poem': 'Literature',
    }

    for key, value in academic_keywords.items():
        if key in q:
            keywords.append(value)

    # ----------------------------------------------------
    # 3) 한국어 키워드 → 분야 매핑
    # ----------------------------------------------------
    keyword_mapping_kr = {
        '심리': 'Psychology',
        '심리학': 'Psychology',
        '사회': 'Sociology',
        '사회학': 'Sociology',
        '통계': 'Statistics',
        '철학': 'Philosophy',
        '역사': 'History',
        '경제': 'Economics',
        '정치': 'Political Science',
        '법': 'Law',
        '의학': 'Medicine',
        '간호': 'Nursing',
        '문학': 'Literature',
    }

    for k, v in keyword_mapping_kr.items():
        if k in question:
            keywords.append(v)

    # ----------------------------------------------------
    # 4) 분야 감지용 트리거 단어 (Law/Psych/Philosophy/History)
    # ----------------------------------------------------
    field_triggers = {
        "law": ["law", "legal", "crime", "contract", "tort", "jurisprudence", "homicide", "larceny"],
        "psych": ["psychology", "behavior", "cognitive", "mental", "disorder", "personality"],
        "phil": ["philosophy", "ethics", "moral", "kant", "aristotle", "epistemology", "metaphysics", "logic"],
        "hist": ["history", "war", "revolution", "ancient", "empire", "dynasty", "ww1", "ww2", "cold war"],
    }

    detected_fields = set()
    for field, triggers in field_triggers.items():
        for t in triggers:
            if t in q:
                detected_fields.add(field)
                break

    # ----------------------------------------------------
    # 5) 대표 Wikipedia 문서 리스트 매핑
    # ----------------------------------------------------
    representative_docs = {
        "law": [
            "Law", "Jurisprudence", "Criminal law", "Contract", "Constitutional law",
            "International law", "Law of war", "Mens rea", "Larceny"
        ],
        "psych": [
            "Psychology", "Cognitive psychology", "Behaviorism",
            "Abnormal psychology", "Personality psychology",
            "Bipolar disorder", "Anxiety disorder", "Classical conditioning"
        ],
        "phil": [
            "Philosophy", "Ethics", "Deontological ethics", "Utilitarianism",
            "Virtue ethics", "Epistemology", "Metaphysics",
            "Immanuel Kant", "Aristotle", "Categorical imperative"
        ],
        "hist": [
            "History", "World history", "French Revolution", "American Revolution",
            "World War I", "World War II", "Cold War", "Industrial Revolution",
            "History of Europe", "History of Asia"
        ],
    }

    for field in detected_fields:
        keywords.extend(representative_docs[field])

    # ----------------------------------------------------
    # 6) 영어 기반 분야 매핑 (추가 감지)
    # ----------------------------------------------------
    english_keywords = [
        'Psychology', 'Education', 'Sociology', 'Statistics', 'Mathematics',
        'Computer Science', 'Biology', 'Chemistry', 'Physics', 'Philosophy',
        'History', 'Literature', 'Economics', 'Political Science', 'Law',
        'Medicine', 'Nursing', 'Pharmacy', 'Art', 'Music'
    ]

    for keyword in english_keywords:
        if keyword.lower() in q:
            keywords.append(keyword)

    # ----------------------------------------------------
    # 7) 중복 제거 & 고유명사 우선순위 정렬
    # ----------------------------------------------------
    keywords = list(set(keywords))

    def priority(k):
        return 0 if k in proper_nouns else 1

    keywords.sort(key=priority)

    return keywords

def extract_choices_from_question(question: str):
    """
    줄바꿈 기반 선택지 추출 (다현 testset.csv 최적화)
    ex)
    (A) text
    (B) text
    (C) text
    ...
    """
    pattern = r"\([A-Z]\)\s*(.+)"
    matches = re.findall(pattern, question)
    return [m.strip() for m in matches if m.strip()]

def extract_keywords_from_choices(choices: List[str]) -> List[str]:
    """
    선택지 텍스트에서 Wikipedia 검색에 사용할 후보 키워드 추출
    """
    keywords = []

    for choice in choices:
        # 예) "Appeal to novelty" → 바로 Wikipedia 검색 가능
        # 예) "Bipolar disorder" → 검색 가능
        # 예) "Straw man" → 검색 가능
        if len(choice.split()) <= 6:  # 너무 긴 건 제거
            keywords.append(choice)

        # 고유명사(예: Kant, Aristotle)
        proper = re.findall(r"\b[A-Z][a-zA-Z]+\b", choice)
        keywords.extend(proper)

    # 중복 제거
    return list(set(keywords))


In [7]:
# ==================================================================
# 4. RAG 파이프라인 구성 및 실행
# ==================================================================

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

# 테스트셋 경로
TESTSET_PATH = CURRENT_DIR / "testset.csv"
TOP_K = 5  # 검색 문서 수 증가


# 리트리버 설정
retriever = vector_store.as_retriever(search_kwargs={"k": TOP_K})

# LLM 및 프롬프트 템플릿 설정
llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2")

prompt_template = PromptTemplate.from_template(
    """
    다음 문맥을 바탕으로 질문에 가장 정확하게 답변하세요.
    
    **중요 지침:**
    1. 이화여대 학칙 문서를 우선 참고하세요 (구체적인 규정)
    2. Wikipedia 문서는 일반적인 학문 지식 참고용입니다
    3. 정확히 하나의 선택지만 답변하세요
    4. 답변 형식: "정답: (X)" 또는 "(X)" 형식으로 답변
    5. 해당 단어가 단순히 포함된게 아니라, 문맥상 정답임을 확신할 때만 선택지로 답변하세요
    
    
    질문이 선택지 형식인 경우, 정답 선택지의 문자(예: (A), (B), (C), (D), (E), (I), (J), (G), (H) 등)를 포함하여 답변하세요.
    최종 답변만 제공하세요.
    
    ---
    Question: {question}
    ---
    Context:
    {context}
    """)
chain = prompt_template | llm

print(f"✅ RAG 체인 구성 완료 (Model: solar-pro2, Top-k: {TOP_K})")

# 테스트셋 로드
def read_data(file_path: Path):
    data = pd.read_csv(file_path)
    return data["prompts"], data.get("answers")

try:
    prompts, answers = read_data(TESTSET_PATH)
    print(f"✅ 테스트 프롬프트 {len(prompts)}개 로드 완료")
except FileNotFoundError:
    print(f"❌ {TESTSET_PATH} 파일을 찾을 수 없습니다.")
    prompts, answers = [], None

# 테스트 실행 (처음 5개만)
if prompts is not None and len(prompts) > 0:
    print("\n▶ 샘플 테스트 시작...\n")
    
    for i, prompt in enumerate(prompts[:5], 1):
        # 1. 기존 벡터 스토어에서 검색
        source_docs = retriever.invoke(prompt)
       # 기존 키워드
        wiki_keywords = extract_wikipedia_keywords(prompt)

        # 1) 선택지 추출
        choices = extract_choices_from_question(prompt)

        # 2) 선택지 기반 Wikipedia 키워드 추출
        choice_keywords = extract_keywords_from_choices(choices)

        # 3) 합치기
        wiki_keywords = list(set(wiki_keywords + choice_keywords))
        
        wiki_docs = []
        if wiki_keywords:
            for keyword in wiki_keywords:
                try:
                    keyword_docs = retrieve_wikipedia_docs(keyword, lang='en')
                    wiki_docs.extend(keyword_docs)
                except Exception as e:
                    # 개별 키워드 검색 실패 시 무시하고 계속 진행
                    pass
        
        # 3. 컨텍스트 생성
        context_parts = []
        if source_docs:
            context_parts.append("=== 이화여대 학칙 문서 ===\n" + "\n\n".join(doc.page_content for doc in source_docs))
        if wiki_docs:
            context_parts.append("=== Wikipedia 문서 ===\n" + "\n\n".join(doc.page_content for doc in wiki_docs))
        
        context_text = "\n\n".join(context_parts) or ""
        
        # 4. 응답 생성
        response = chain.invoke({"question": prompt, "context": context_text})
        
        print(f"Q{i}: {prompt[:60]}...")
        print(f"응답: {response.content[:150]}...")
        print(f"검색 문서: 이화여대 {len(source_docs)}개, Wikipedia {len(wiki_docs)}개 (키워드: {', '.join(wiki_keywords) if wiki_keywords else '없음'})\n")
    
    print("✅ 샘플 테스트 완료")
else:
    print("⚠️ 실행할 프롬프트가 없습니다.")


✅ RAG 체인 구성 완료 (Model: solar-pro2, Top-k: 5)
✅ 테스트 프롬프트 50개 로드 완료

▶ 샘플 테스트 시작...

Q1: QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 휴학을 신청하야하나요?
(...
응답: 정답: (D) 90일

Context에서 명시된 학칙 제26조 ⑥항에 따르면, 재학 중인 학생이 휴학을 하고자 하는 경우 "학기개시일로부터 90일 이내에 휴학을 신청하여야 한다"고 규정되어 있습니다. 따라서 정답은 (D)입니다....
검색 문서: 이화여대 5개, Wikipedia 0개 (키워드: 60일, 90일, 30일, 45일)

Q2: QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 의하여 제적된 자는 제적된...
응답: 정답: (B) 3

**근거:**
- **a 값:** "재입학은 1회에 한하여 할 수 있다" (제30조 제2항) → a = 1
- **b 값:** "제28조제4호에 의하여 제적된 자는 제적된 날부터 1년이 경과한 후 재입학할 수 있다" (제28조 제4호) → b = 1...
검색 문서: 이화여대 5개, Wikipedia 30개 (키워드: 4, A,B,C 중 답 없음, 3, 2)

Q3: QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 몇학점 이상 ...
응답: 정답: (C) 21학점

이화여대 학칙 문서 6.23) ③항에 명시된 내용에 따르면, "학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 21학점 이상 취득한 때에는 부전공을 이수한 것으로 인정한다"고 규정되어 있습니다. 따라서 정확한 답변은 ...
검색 문서: 이화여대 5개, Wikipedia 0개 (키워드: 18학점, 25학점, 15학점, 21학점)

Q4: QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?
(A) 팜 : 징계에 의해 퇴...
응답: 정답: (D)  

**해

In [8]:
# ==================================================================
# 5. 전체 테스트셋 평가 (다현님 + 나현님 방식 결합)
# ==================================================================

def extract_answer(response: str) -> Optional[str]:
    """
    응답에서 답변 추출 개선 버전 (모든 알파벳 선택지 지원: A-Z)
    - 여러 선택지가 있을 경우 가장 확실한 것 선택
    - "정답:", "Answer:", "답:" 등의 키워드 우선
    """
    # 우선순위 1: 명시적인 정답 표시 (한국어/영어)
    explicit_patterns = [
        r"정답[:\s]*\(([A-Z])\)",
        r"Answer[:\s]*\(([A-Z])\)",
        r"답[:\s]*\(([A-Z])\)",
        r"\[ANSWER\]:\s*\(([A-Z])\)",
        r"최종\s*답[:\s]*\(([A-Z])\)",
        r"Final\s*Answer[:\s]*\(([A-Z])\)",
    ]
    for pattern in explicit_patterns:
        match = re.search(pattern, response, re.IGNORECASE)
        if match:
            return match.group(1).upper()
    
    # 우선순위 2: (X) 형식 (마지막 발생) - 모든 알파벳 지원
    # 여러 개가 있으면 마지막 것 (보통 최종 답변)
    pattern2 = r"\(([A-Z])\)"
    matches = list(re.finditer(pattern2, response))
    if matches:
        return matches[-1].group(1)
    
    # 우선순위 3: 단독 문자 (마지막 발생) - 모든 알파벳 지원
    # 단어 경계에서 알파벳만 매칭 (대소문자 구분 없음)
    pattern3 = r"\b([A-Z])\b"
    matches = list(re.finditer(pattern3, response, re.IGNORECASE))
    if matches:
        return matches[-1].group(1).upper()
    
    return None

# 전체 평가 실행
if prompts is not None and len(prompts) > 0:
    print("====== RAG Evaluation (전체 테스트셋) ======\n")
    print(f"📋 총 {len(prompts)}개 문제 평가 시작...\n")
    
    all_responses = []
    predictions = []
    
    for i, prompt in enumerate(prompts, 1):
        try:
            print(f"🔄 Q{i}/{len(prompts)} 처리 중...", end=" ")
            
            # 1. 기존 벡터 스토어에서 검색
            source_docs = retriever.invoke(prompt)
            
            # 2. Wikipedia 검색 키워드 생성 (선택지 기반 + 분야 감지 + 고유명사 기반)
            base_keywords = extract_wikipedia_keywords(prompt)

            # 선택지 기반 키워드 추출
            choices = extract_choices_from_question(prompt)
            choice_keywords = extract_keywords_from_choices(choices)

            # 최종 키워드 합치기
            wiki_keywords = list(set(base_keywords + choice_keywords))  
            # Wikipedia 문서 검색
            wiki_docs = []
            for keyword in wiki_keywords:
                try:
                    wiki_docs.extend(retrieve_wikipedia_docs(keyword, lang='en'))
                except:
                    pass
            # 위키 문서가 너무 많을 경우 10개 제한
            if len(wiki_docs) > 10:
                wiki_docs = wiki_docs[:10]
            
            # 3. 기존 문서와 Wikipedia 문서 통합
            all_source_docs = source_docs + wiki_docs
            
            # 4. 컨텍스트 생성
            context_parts = []
            if source_docs:
                context_parts.append("=== 이화여대 학칙 문서 ===\n" + "\n\n".join(doc.page_content for doc in source_docs))
            if wiki_docs:
                context_parts.append("=== Wikipedia 문서 ===\n" + "\n\n".join(doc.page_content for doc in wiki_docs))
            
            context_text = "\n\n".join(context_parts) or ""
            
            # 5. 응답 생성
            response = chain.invoke({"question": prompt, "context": context_text})
            
            pred_raw = response.content.strip()
            pred = extract_answer(pred_raw)
            
            predictions.append(pred)
            all_responses.append({
                "question_num": i,
                "question": prompt[:100],
                "prediction": pred,
                "response": pred_raw[:200],
                "context_count": len(source_docs),
                "wikipedia_count": len(wiki_docs),
                "wikipedia_keywords": ", ".join(wiki_keywords) if wiki_keywords else ""
            })
            
            # 진행 상황 출력 (모든 문제에 대해)
            gold = answers.iloc[i-1] if answers is not None else "N/A"
            # Gold 답변에서 선택지 문자 추출 (예: (D) -> D)
            gold_match = re.search(r"\(([A-Z])\)", str(gold))
            gold_letter = gold_match.group(1) if gold_match else str(gold).strip()
            
            # 정답 여부 확인 (대소문자 구분 없이)
            is_correct = pred and str(pred).upper() == str(gold_letter).upper()
            status = "✅" if is_correct else "❌" if pred else "⚠️"
            print(f"{status} Pred: {pred}, Gold: {gold} ({gold_letter})")
            
            # 10개마다 상세 정보 출력
            if i % 10 == 0 or i == len(prompts):
                print(f"  └─ 진행률: {i}/{len(prompts)} ({i/len(prompts)*100:.1f}%)")
                print("-" * 40)
        
        except Exception as e:
            import traceback
            error_msg = str(e)
            error_trace = traceback.format_exc()
            print(f"❌ 에러 발생: {error_msg}")
            print(f"  └─ Q{i} 처리 실패, None으로 저장")
            if i == 43:  # 43번 문제 디버깅
                print(f"  └─ 상세 에러:\n{error_trace}")
            predictions.append(None)
            all_responses.append({
                "question_num": i,
                "question": prompt[:100] if prompt else "",
                "prediction": None,
                "response": f"Error: {error_msg}",
                "context_count": 0,
                "wikipedia_count": 0,
                "wikipedia_keywords": ""
            })
            # 에러가 발생해도 계속 진행
            continue
    
    # 결과 저장
    results_df = pd.DataFrame(all_responses)
    results_path = CURRENT_DIR / "rag_results_combined.csv"
    results_df.to_csv(results_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ 결과 저장 완료: {results_path}")
    
    # 정확도 계산
    if answers is not None:
        correct = 0
        total = 0
        for i, (pred, gold) in enumerate(zip(predictions, answers)):
            if pred is None:
                continue
            gold_match = re.search(r"\(([A-Z])\)", str(gold))
            gold_letter = gold_match.group(1) if gold_match else str(gold).strip()
            if str(pred).upper() == str(gold_letter).upper():
                correct += 1
            total += 1
        
        if total > 0:
            accuracy = correct / total * 100
            print(f"\n📊 정확도: {correct}/{total} ({accuracy:.1f}%)")
            print(f"   - 정답: {correct}개")
            print(f"   - 오답: {total - correct}개")
            print(f"   - 미응답: {len(predictions) - total}개")
else:
    print("⚠️ 실행할 프롬프트가 없습니다.")


====== RAG Evaluation (전체 테스트셋) ======

📋 총 50개 문제 평가 시작...

🔄 Q1/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q2/50 처리 중... ✅ Pred: A, Gold: (A) (A)
🔄 Q3/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q4/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q5/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q6/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q7/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q8/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q9/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q10/50 처리 중... ✅ Pred: B, Gold: (B) (B)
  └─ 진행률: 10/50 (20.0%)
----------------------------------------
🔄 Q11/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q12/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q13/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q14/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q15/50 처리 중... ✅ Pred: A, Gold: (A) (A)
🔄 Q16/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q17/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q18/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q19/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q20/50 처리 중... ✅ Pred: D, Gold: (D) (D)
  └─ 진행률: 20/50 (40.0%)
-------------------